In [28]:
from pymongo import MongoClient
import pymongo
import json
import time
from mysql.connector import (connection)
import mysql.connector

class ProjectCache:
    
    def __init__(self, database):
        self.cache = {}                                      #Cache
        self.minheap = []                                    #Minheap
        self.cache_limit_value = 200                         #The cache will be set up to only hold 200 tweets
        self.db = MongoClient(database)["TwitterDB"]
        self.tweets = self.db["tweets"]
        ktdm = list(self.db["keywords"].find())  #Holds all keywords and ids associated
        self.keyword_tdm = dict((x['_id'], x['tweets']) for x in ktdm)
        htdm = list(self.db["hashtags"].find())
        self.hashtag_tdm = dict((x['_id'], x['tweets']) for x in htdm)              #Holds all hashtags and ids associated
        self.recent_keyword_searches_cache = {}                      
        self.recent_hashtag_searches_cache = {}
        self.total_searches_kept_in_cache = 3                        
        self.recent_keyword_searches = []
        self.recent_hashtag_searches = []
        self.connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
        
        
    def __contains__(self, key):
        
        return key in self.cache
    
    def current_cache_size(self):
        
        return len(self.cache)
    
    def initial_population(self):
        
        #Read every tweet in collection and attempt to add them one by one to cache
        for i in self.tweets:
            add_update(i["id"], i)
           
    def add_update(self, key, value):
        
        #Add a section that first checks if the current key's relevance value is greater 
        #than the least relevant term in the cache
        if key not in self.cache:
            #If the cache limit is met get the least relevant in cache then compare current item with least relevant in cache
            if len(self.cache) >= self.cache_limit_value:  
                least_relevant_term()                                       
                if value["relevance"] > least_relevant_value_in_cache:      
                    
                    #then remove least relevant to make room in cache
                    self.remove_least_relevant(least_relevant_key, least_relevant_value_in_cache)          
                    
                    #Add key and relevance value to minheap list
                    minheap_sort(key, value["relevance"])
                    
                    #Then add key and all tweet elements to cache
                    self.cache[key] = value                  
        
            else:
                #Add key and relevance value to minheap list
                minheap_sort(key, value["relevance"])
                
                #Then add key and all tweet elements to cache
                self.cache[key] = value
        
    def remove_least_relevant(self, least_relevant_key, least_relevant_value_in_cache):
        
        #Remove least relevant from both the cache and the heaped list
        self.cache.pop(least_relevant)
        heappop(self.minheap, (least_relevant_value_in_cache, least_relevant))
        heapify(self.minheap)
        
    def least_relevant_term(self):
        
        #Get the least relevant term within the cache as well as key associated with it.      
        least_relevant_value_in_cache = self.minheap[0][0]
        least_relevant_key = self.minheap[0][1]
        return least_relevant_value_in_cache, least_relevant_key
    
    def minheap_sort(self, key, value):
        
        #The relevance will be the new key, and the key will be the value in the minheap.      
        heappush(self.minheap, (value, key))
        heapify(self.minheap)
    
    def userlist(self, l):
        
        userCounts = l.user.value_counts().to_frame(name = 'relevant_tweets')
        userCounts['user_id'] = list(userCounts.index)
        
        print('u1')
        print(time.time())
        
        query = """
        SELECT *
        FROM new_database.users_table 
        WHERE id IN (%s)
        """ % ', '.join(str(x) for x in userCounts.user_id)
        
        print('u2')
        print(time.time())
        
        userList = pd.read_sql(query, connection1)
        
        print('u3')
        print(time.time())
        
        userCounts = userCounts.merge(userList, left_on = 'user_id', right_on = 'id')
        
        userCounts = userCounts.sort_values(by=['relevant_tweets', 'followers_count', 'friends_count'], ascending = False)
        
        print('u4')
        print(time.time())
        
        return userCounts.head(5)
        
    
    def keyword(self, word, querytype):
        
        #Make a blank dictionary
        l={}
        
        #Grab all the tweet ids associated with the word/hashtag
        if querytype == "Text":
            #If word is in searched cache just return that dictionary
            if word in self.recent_keyword_searches_cache:
                self.recent_keyword_searches.remove(word)
                self.recent_keyword_searches.append(word)
                l = self.recent_keyword_searches_cache[word]
                return pd.DataFrame.from_dict(l).transpose()
            else:
                ids = self.keyword_tdm[word] #Could use .head(10) to get only the top 10
        elif querytype == "Hashtag":
            #If word is in searched cache just return that dictionary
            if word in self.recent_hashtag_searches_cache:
                self.recent_hashtag_searches.remove(word)
                self.recent_hashtag_searches.append(word)
                l = self.recent_hashtag_searches_cache[word]
                return pd.DataFrame.from_dict(l).transpose()
            else:
                ids = self.hashtag_tdm[word]
        
        #Search through cache first for all tweet ids
        #If it's not in the cache find it in the database
        for i in ids:
            if i in self.cache:
                l[i] = self.cache[i]
            else:
                l[i] = self.tweets.find_one({"_id":i})
                
        #make a nested dictionary with the three most recent searched words saved in a queue
        if querytype == "Text":
            self.recent_keyword_searches_cache[word] = l
            self.recent_keyword_searches.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.recent_keyword_searches_cache) > self.total_searches_kept_in_cache:
                a = self.recent_keyword_searches.pop(0)
                self.recent_keyword_searches_cache.pop(a)
        elif querytype == "Hashtag":
            self.recent_hashtag_searches_cache[word] = l
            self.recent_hashtag_searches.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.recent_hashtag_searches_cache) > self.total_searches_kept_in_cache:
                a = self.recent_hashtag_searches.pop(0)
                self.recent_hashtag_searches_cache.pop(a)
        
        l = pd.DataFrame.from_dict(l).transpose()
        
        #Return a dictionary of all of the ids and associated tweets
        return l
    



In [29]:
from dash import Dash, html, dcc, callback, Output, Input, dash_table
from dash.dependencies import State
import pandas as pd

# import dash_bootstrap_components as dbc

# Some default set of tweets (could be 1) - the relevant part is the column headers
#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

database = "mongodb://localhost:27017"

PC = ProjectCache(database)

In [ ]:
from dash import Dash, html, dcc, callback
from dash.dependencies import Input, Output, State
import dash_table
import pandas as pd
import time

search_output_columns = [{"name": i, "id": i, "type": "text", "presentation": "markdown"} for i in ['user_name', 'created_at', 'text', 'reply_count', 'retweet_count']]
search_output_columns[2]['style'] = {'maxWidth': '200px', 'overflow': 'hidden', 'textOverflow': 'ellipsis'}

user_df_columns = [{"name": i, "id": i, "type": "text", "presentation": "markdown"} for i in ['name', 'screen_name', 'relevant_tweets', 'created_at', 'followers_count', 'friends_count', 'verified']]
user_df_columns[2]['style'] = {'maxWidth': '200px', 'overflow': 'hidden', 'textOverflow': 'ellipsis'}

app.layout = html.Div([
    html.H1(children='Tweet Search', style={'textAlign':'center'}),
    dcc.RadioItems(['Text', 'Hashtag'], 'Text', id='search_selection', inline=True),
    dcc.Input(id="search_query", type="text", placeholder="search query"),
    html.Button('Search', id='search_submit'),
    html.Div(id='search_time'),
    html.H2(children='Output'),
    dash_table.DataTable(
        data=[],
        columns=search_output_columns,
        id='search_output',
        style_cell={'whiteSpace': 'normal', 'height': 'auto'},
        style_data_conditional=[{'whiteSpace': 'normal', 'height': 'auto'}]
    ),
    html.H2(children='Users'),
    dash_table.DataTable(
        data=pd.DataFrame().to_dict('records'),
        columns=user_df_columns,
        id='user_df',
        style_cell={'whiteSpace': 'normal', 'height': 'auto'},
        style_data_conditional=[{'whiteSpace': 'normal', 'height': 'auto'}]
    )
])


@callback(
    Output('search_output', 'data'),
    Output('user_df', 'data'),
    Output('search_time', 'children'),
    State('search_selection', 'value'),
    State('search_query', 'value'),
    Input('search_submit', 'n_clicks')
) # This function should be updated to refer to and call tweets as appropriate
def update_table(stype, value, n):
    start = time.time()
    print(start)
    dff = PC.keyword(value, stype)
    print('Got tweets')
    print(time.time())
    dff2 = PC.userlist(dff)
    print('Got users')
    print(time.time())
    
    end = time.time()
    return dff.reset_index().to_dict("records"), dff2.reset_index().to_dict("records"), "Search Time: {} s".format(end-start, 3)

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2023 00:59:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 00:59:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 00:59:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 00:59:42] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
[2023-04-28 00:59:42,431] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/mazinrafi/miniconda3/lib/python3.10/site-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/mazinrafi/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/mazinrafi/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/mazinrafi/miniconda3/lib/python

1682657982.4287202


/var/folders/tk/17szgjb53c52s1rvb1qj14q00000gn/T/ipykernel_78598/2543684043.py:105: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

127.0.0.1 - - [28/Apr/2023 00:59:44] "POST /_dash-update-component HTTP/1.1" 200 -


1682657984.429358
Got tweets
1682657984.4439502
u1
1682657984.451583
u2
1682657984.4517472
u3
1682657984.485827
u4
1682657984.493749
Got users
1682657984.493818


/var/folders/tk/17szgjb53c52s1rvb1qj14q00000gn/T/ipykernel_78598/2543684043.py:105: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

127.0.0.1 - - [28/Apr/2023 00:59:59] "POST /_dash-update-component HTTP/1.1" 200 -


1682657999.675273
Got tweets
1682657999.6945999
u1
1682657999.697773
u2
1682657999.698171
u3
1682657999.732918
u4
1682657999.7369988
Got users
1682657999.737157
